# Отбор признаков: классификация методов

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

### Загрузка данных

In [37]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [39]:
data = pd.read_excel('data/data_ford_price.xlsx') 

# удалим пропуски
data.dropna(inplace=True)

# удалим столбцы типа object
data.drop(columns=['title_status', 'transmission', 'drive', 'size'], inplace=True)

# выделим целевой признак
y = data['price']
X = data.drop(columns='price')

In [40]:
# разобьем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

##  Отбор признаков. Метод рекурсивного исключения признаков (RFE)

In [41]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

In [42]:
X_train.columns

Index(['year', 'condition', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [43]:
selector.ranking_

array([1, 1, 1, 5, 2, 4, 3])

### Обучим модель


In [44]:
X_train_rfe = X_train[['year', 'condition', 'cylinders']]
X_test_rfe = X_test[['year', 'condition', 'cylinders']]

In [45]:
lr_model = LinearRegression()
lr_model.fit(X_train_rfe, y_train)
y_train_predict = lr_model.predict(X_train_rfe)
y_test_predict = lr_model.predict(X_test_rfe)

print("Train R^2: {:.3f}".format(r2_score(y_train, y_train_predict)))
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict)))

Train R^2: 0.572
Test R^2: 0.594


##  Отбор признаков. Метод выбора признаков на основе фильтров

In [46]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

### Обучим модель


In [47]:
X_train_k = X_train[['year', 'condition', 'odometer']]
X_test_k = X_test[['year', 'condition', 'odometer']]

In [48]:
lr_model = LinearRegression()
lr_model.fit(X_train_k, y_train)
y_train_predict = lr_model.predict(X_train_k)
y_test_predict = lr_model.predict(X_test_k)

print("Train R^2: {:.3f}".format(r2_score(y_train, y_train_predict)))
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict)))

Train R^2: 0.601
Test R^2: 0.636


Вывод:

 Метрика во втором случае выше, чем в первом (0.636 > 0.594 - показатели метрики r2 score на тестовой выборке). Значит, в данном случаем метод выбора признаков на основе коэффициента Пирсона показал себя лучше, чем метод рекурсивного исключения признаков. 